# 🎯 Cancer Prediction - Optimized ML Pipeline

## Βελτιστοποιημένο Notebook για Πρόβλεψη Καρκίνου

**Dataset**: Wisconsin Breast Cancer Dataset (569 samples, 30 features)

**Στόχος**: Binary Classification - Διάγνωση (M=Malignant/Κακοήθης, B=Benign/Καλοήθης)

**Βελτιώσεις**:
- ✅ Fixed data leakage issue
- ✅ Cross-validation implementation
- ✅ Multiple ML models comparison
- ✅ Hyperparameter tuning
- ✅ Feature selection & importance analysis
- ✅ Enhanced visualizations (ROC, confusion matrices, etc.)
- ✅ Model persistence

## 📚 Step 1: Import Libraries

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning - Preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif, RFE

# Machine Learning - Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

# Machine Learning - Metrics
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, roc_auc_score, precision_recall_curve,
    f1_score, precision_score, recall_score
)

# Model Persistence
import joblib

# Settings
import warnings
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported successfully!")

## 📊 Step 2: Load Data

In [ ]:
# Load dataset
url = 'https://raw.githubusercontent.com/ybifoundation/Dataset/main/Cancer.csv'
cancer = pd.read_csv(url)

print(f"✅ Dataset loaded successfully!")
print(f"Shape: {cancer.shape}")
print(f"\nFirst 5 rows:")
cancer.head()

## 🔍 Step 3: Exploratory Data Analysis (EDA)

In [ ]:
# Dataset info
print("📋 Dataset Information:")
print(cancer.info())
print("\n" + "="*80)

# Statistical summary
print("\n📊 Statistical Summary:")
print(cancer.describe())
print("\n" + "="*80)

# Missing values
print("\n❓ Missing Values:")
missing = cancer.isnull().sum()
print(missing[missing > 0] if missing.sum() > 0 else "No missing values!")
print("\n" + "="*80)

# Target distribution
print("\n🎯 Target Distribution:")
print(cancer['diagnosis'].value_counts())
print(f"\nClass Balance: {cancer['diagnosis'].value_counts(normalize=True) * 100}")

In [ ]:
# Visualize target distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Count plot
cancer['diagnosis'].value_counts().plot(kind='bar', ax=axes[0], color=['#2ecc71', '#e74c3c'])
axes[0].set_title('Diagnosis Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Diagnosis (B=Benign, M=Malignant)')
axes[0].set_ylabel('Count')
axes[0].set_xticklabels(['Benign', 'Malignant'], rotation=0)

# Pie chart
cancer['diagnosis'].value_counts().plot(kind='pie', ax=axes[1], autopct='%1.1f%%', 
                                         colors=['#2ecc71', '#e74c3c'], startangle=90)
axes[1].set_title('Diagnosis Proportion', fontsize=14, fontweight='bold')
axes[1].set_ylabel('')

plt.tight_layout()
plt.show()

## 🧹 Step 4: Data Preprocessing

In [ ]:
# Remove unnecessary columns
columns_to_drop = ['id', 'Unnamed: 32'] if 'Unnamed: 32' in cancer.columns else ['id']
cancer_clean = cancer.drop(columns=columns_to_drop)

# Define features and target
X = cancer_clean.drop('diagnosis', axis=1)
y = cancer_clean['diagnosis']

print(f"✅ Features shape: {X.shape}")
print(f"✅ Target shape: {y.shape}")
print(f"\nFeatures: {list(X.columns)}")

In [ ]:
# Correlation heatmap for top features
plt.figure(figsize=(16, 14))
correlation_matrix = X.corr()
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Heatmap', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Find highly correlated features
high_corr = np.where(np.abs(correlation_matrix) > 0.9)
high_corr_list = [(correlation_matrix.index[x], correlation_matrix.columns[y], correlation_matrix.iloc[x, y]) 
                  for x, y in zip(*high_corr) if x != y and x < y]
print(f"\n🔗 Highly Correlated Features (>0.9): {len(high_corr_list)} pairs found")

## 🔀 Step 5: Train-Test Split (FIXED - No Data Leakage!)

In [ ]:
# Split data BEFORE any scaling or preprocessing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"✅ Train set: {X_train.shape} | Test set: {X_test.shape}")
print(f"\nTrain class distribution:\n{y_train.value_counts()}")
print(f"\nTest class distribution:\n{y_test.value_counts()}")

## ⚖️ Step 6: Feature Scaling (CORRECTED - After Split!)

In [ ]:
# Initialize scaler
scaler = StandardScaler()

# FIT on training data ONLY, then transform both
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)  # Only transform, no fit!

# Convert back to DataFrame for better readability
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns, index=X_test.index)

print("✅ Feature scaling completed correctly (NO data leakage!)")
print(f"\nScaled training data sample:")
print(X_train_scaled.head())

## 🎯 Step 7: Feature Selection

In [ ]:
# Method 1: SelectKBest (ANOVA F-test)
selector = SelectKBest(score_func=f_classif, k=15)
X_train_selected = selector.fit_transform(X_train_scaled, y_train)
X_test_selected = selector.transform(X_test_scaled)

# Get selected feature names
selected_features = X.columns[selector.get_support()].tolist()
feature_scores = pd.DataFrame({
    'Feature': X.columns,
    'Score': selector.scores_
}).sort_values('Score', ascending=False)

print(f"✅ Top 15 features selected:")
print(selected_features)
print(f"\n📊 Feature Importance Scores:")
print(feature_scores.head(15))

In [ ]:
# Visualize feature importance
plt.figure(figsize=(12, 8))
top_features = feature_scores.head(15)
sns.barplot(data=top_features, y='Feature', x='Score', palette='viridis')
plt.title('Top 15 Most Important Features (ANOVA F-Score)', fontsize=14, fontweight='bold')
plt.xlabel('F-Score')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()

## 🤖 Step 8: Baseline Model Comparison

In [ ]:
# Define multiple models to compare
models = {
    'Logistic Regression': LogisticRegression(max_iter=5000, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM (RBF)': SVC(kernel='rbf', probability=True, random_state=42),
    'SVM (Linear)': SVC(kernel='linear', probability=True, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
    'Naive Bayes': GaussianNB()
}

# Cross-validation setup
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Store results
results = []

print("🔄 Training and evaluating models with 5-fold cross-validation...\n")

for name, model in models.items():
    # Cross-validation scores
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=cv, scoring='accuracy')
    
    # Train on full training set
    model.fit(X_train_scaled, y_train)
    
    # Predictions
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1] if hasattr(model, 'predict_proba') else None
    
    # Metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, pos_label='M')
    recall = recall_score(y_test, y_pred, pos_label='M')
    f1 = f1_score(y_test, y_pred, pos_label='M')
    
    results.append({
        'Model': name,
        'CV Mean': cv_scores.mean(),
        'CV Std': cv_scores.std(),
        'Test Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1
    })
    
    print(f"✅ {name}:")
    print(f"   CV Accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")
    print(f"   Test Accuracy: {accuracy:.4f}")
    print(f"   F1-Score: {f1:.4f}\n")

# Create results DataFrame
results_df = pd.DataFrame(results).sort_values('Test Accuracy', ascending=False)
print("\n" + "="*80)
print("📊 Model Comparison Summary:")
print(results_df.to_string(index=False))

In [ ]:
# Visualize model comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Test Accuracy
sns.barplot(data=results_df, x='Test Accuracy', y='Model', ax=axes[0, 0], palette='Blues_r')
axes[0, 0].set_title('Test Accuracy Comparison', fontsize=12, fontweight='bold')
axes[0, 0].set_xlim(0.9, 1.0)

# Cross-Validation Mean
sns.barplot(data=results_df, x='CV Mean', y='Model', ax=axes[0, 1], palette='Greens_r')
axes[0, 1].set_title('Cross-Validation Accuracy', fontsize=12, fontweight='bold')
axes[0, 1].set_xlim(0.9, 1.0)

# Precision
sns.barplot(data=results_df, x='Precision', y='Model', ax=axes[1, 0], palette='Oranges_r')
axes[1, 0].set_title('Precision Score', fontsize=12, fontweight='bold')
axes[1, 0].set_xlim(0.9, 1.0)

# F1-Score
sns.barplot(data=results_df, x='F1-Score', y='Model', ax=axes[1, 1], palette='Reds_r')
axes[1, 1].set_title('F1-Score Comparison', fontsize=12, fontweight='bold')
axes[1, 1].set_xlim(0.9, 1.0)

plt.tight_layout()
plt.show()

## 🎯 Step 9: Hyperparameter Tuning (Best Models)

In [ ]:
# Get top 3 models for hyperparameter tuning
top_3_models = results_df.head(3)['Model'].tolist()
print(f"🎯 Tuning hyperparameters for top 3 models: {top_3_models}\n")

# Define parameter grids
param_grids = {
    'Logistic Regression': {
        'C': [0.01, 0.1, 1, 10, 100],
        'penalty': ['l2'],
        'solver': ['lbfgs', 'liblinear']
    },
    'Random Forest': {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'SVM (RBF)': {
        'C': [0.1, 1, 10, 100],
        'gamma': ['scale', 'auto', 0.001, 0.01, 0.1]
    },
    'SVM (Linear)': {
        'C': [0.1, 1, 10, 100]
    },
    'Gradient Boosting': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7]
    }
}

# Tune top models
tuned_models = {}

for name in top_3_models:
    if name in param_grids:
        print(f"\n🔧 Tuning {name}...")
        
        model = models[name]
        param_grid = param_grids[name]
        
        grid_search = GridSearchCV(
            model, param_grid, cv=cv, scoring='accuracy', 
            n_jobs=-1, verbose=0
        )
        
        grid_search.fit(X_train_scaled, y_train)
        
        tuned_models[name] = grid_search.best_estimator_
        
        print(f"   ✅ Best params: {grid_search.best_params_}")
        print(f"   ✅ Best CV score: {grid_search.best_score_:.4f}")
        print(f"   ✅ Test accuracy: {grid_search.score(X_test_scaled, y_test):.4f}")

print("\n" + "="*80)
print("✅ Hyperparameter tuning completed!")

## 🏆 Step 10: Select Best Model

In [ ]:
# Evaluate tuned models
best_score = 0
best_model_name = None
best_model = None

for name, model in tuned_models.items():
    score = model.score(X_test_scaled, y_test)
    if score > best_score:
        best_score = score
        best_model_name = name
        best_model = model

print(f"🏆 Best Model: {best_model_name}")
print(f"🎯 Test Accuracy: {best_score:.4f} ({best_score*100:.2f}%)")
print(f"\n📋 Model Parameters:")
print(best_model.get_params())

## 📊 Step 11: Detailed Evaluation of Best Model

In [ ]:
# Predictions
y_pred = best_model.predict(X_test_scaled)
y_pred_proba = best_model.predict_proba(X_test_scaled)[:, 1] if hasattr(best_model, 'predict_proba') else None

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred, labels=['B', 'M'])

# Classification Report
print("📋 Classification Report:")
print(classification_report(y_test, y_pred, target_names=['Benign', 'Malignant']))

# Detailed metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label='M')
recall = recall_score(y_test, y_pred, pos_label='M')
f1 = f1_score(y_test, y_pred, pos_label='M')

print("\n" + "="*80)
print("📊 Performance Metrics:")
print(f"   Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"   Precision: {precision:.4f} ({precision*100:.2f}%)")
print(f"   Recall:    {recall:.4f} ({recall*100:.2f}%)")
print(f"   F1-Score:  {f1:.4f} ({f1*100:.2f}%)")

if y_pred_proba is not None:
    auc_score = roc_auc_score(y_test.map({'B': 0, 'M': 1}), y_pred_proba)
    print(f"   AUC-ROC:   {auc_score:.4f} ({auc_score*100:.2f}%)")

In [ ]:
# Visualize Confusion Matrix
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Count Confusion Matrix
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0], 
            xticklabels=['Benign', 'Malignant'], yticklabels=['Benign', 'Malignant'])
axes[0].set_title(f'Confusion Matrix - {best_model_name}\n(Counts)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('True Label')
axes[0].set_xlabel('Predicted Label')

# Normalized Confusion Matrix
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Greens', ax=axes[1],
            xticklabels=['Benign', 'Malignant'], yticklabels=['Benign', 'Malignant'])
axes[1].set_title(f'Confusion Matrix - {best_model_name}\n(Normalized)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('True Label')
axes[1].set_xlabel('Predicted Label')

plt.tight_layout()
plt.show()

# Print confusion matrix interpretation
tn, fp, fn, tp = cm.ravel()
print(f"\n🔍 Confusion Matrix Breakdown:")
print(f"   True Negatives (Benign correctly identified):  {tn}")
print(f"   False Positives (Benign predicted as Malignant): {fp}")
print(f"   False Negatives (Malignant predicted as Benign): {fn}")
print(f"   True Positives (Malignant correctly identified): {tp}")

In [ ]:
# ROC Curve and Precision-Recall Curve
if y_pred_proba is not None:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # ROC Curve
    y_test_binary = y_test.map({'B': 0, 'M': 1})
    fpr, tpr, thresholds_roc = roc_curve(y_test_binary, y_pred_proba)
    auc_score = roc_auc_score(y_test_binary, y_pred_proba)
    
    axes[0].plot(fpr, tpr, linewidth=2, label=f'ROC Curve (AUC = {auc_score:.4f})')
    axes[0].plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')
    axes[0].set_xlabel('False Positive Rate')
    axes[0].set_ylabel('True Positive Rate')
    axes[0].set_title(f'ROC Curve - {best_model_name}', fontsize=12, fontweight='bold')
    axes[0].legend(loc='lower right')
    axes[0].grid(True, alpha=0.3)
    
    # Precision-Recall Curve
    precision_curve, recall_curve, thresholds_pr = precision_recall_curve(y_test_binary, y_pred_proba)
    
    axes[1].plot(recall_curve, precision_curve, linewidth=2, label=f'{best_model_name}')
    axes[1].axhline(y=y_test_binary.mean(), color='k', linestyle='--', linewidth=1, label='Baseline')
    axes[1].set_xlabel('Recall')
    axes[1].set_ylabel('Precision')
    axes[1].set_title(f'Precision-Recall Curve - {best_model_name}', fontsize=12, fontweight='bold')
    axes[1].legend(loc='lower left')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 🎨 Step 12: Feature Importance Analysis

In [ ]:
# Feature importance (for tree-based models)
if hasattr(best_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'Feature': X.columns,
        'Importance': best_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print("📊 Top 15 Most Important Features:")
    print(feature_importance.head(15))
    
    # Visualize
    plt.figure(figsize=(12, 8))
    sns.barplot(data=feature_importance.head(15), y='Feature', x='Importance', palette='rocket')
    plt.title(f'Top 15 Feature Importances - {best_model_name}', fontsize=14, fontweight='bold')
    plt.xlabel('Importance Score')
    plt.ylabel('Feature')
    plt.tight_layout()
    plt.show()
    
elif hasattr(best_model, 'coef_'):
    # For linear models (Logistic Regression, SVM Linear)
    coef = best_model.coef_[0] if best_model.coef_.ndim > 1 else best_model.coef_
    feature_importance = pd.DataFrame({
        'Feature': X.columns,
        'Coefficient': coef,
        'Abs_Coefficient': np.abs(coef)
    }).sort_values('Abs_Coefficient', ascending=False)
    
    print("📊 Top 15 Most Important Features (by coefficient magnitude):")
    print(feature_importance.head(15))
    
    # Visualize
    plt.figure(figsize=(12, 8))
    top_15 = feature_importance.head(15)
    colors = ['red' if x < 0 else 'green' for x in top_15['Coefficient']]
    sns.barplot(data=top_15, y='Feature', x='Coefficient', palette=colors)
    plt.title(f'Top 15 Feature Coefficients - {best_model_name}', fontsize=14, fontweight='bold')
    plt.xlabel('Coefficient Value')
    plt.ylabel('Feature')
    plt.axvline(x=0, color='black', linestyle='--', linewidth=1)
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ Feature importance not available for this model type.")

## 💾 Step 13: Save Model & Scaler

In [ ]:
# Save the best model
model_filename = 'best_cancer_model.pkl'
scaler_filename = 'feature_scaler.pkl'

joblib.dump(best_model, model_filename)
joblib.dump(scaler, scaler_filename)

print(f"✅ Model saved as '{model_filename}'")
print(f"✅ Scaler saved as '{scaler_filename}'")

# Save model metadata
metadata = {
    'model_name': best_model_name,
    'test_accuracy': best_score,
    'features': list(X.columns),
    'target_classes': ['B', 'M']
}

print(f"\n📋 Model Metadata:")
for key, value in metadata.items():
    if key != 'features':
        print(f"   {key}: {value}")

## 🔮 Step 14: Example Prediction with Loaded Model

In [ ]:
# Load model (demonstration)
loaded_model = joblib.load(model_filename)
loaded_scaler = joblib.load(scaler_filename)

print("✅ Model and scaler loaded successfully!")

# Make prediction on a sample
sample = X_test.iloc[0:1]
sample_scaled = loaded_scaler.transform(sample)
prediction = loaded_model.predict(sample_scaled)
prediction_proba = loaded_model.predict_proba(sample_scaled) if hasattr(loaded_model, 'predict_proba') else None

print(f"\n🔮 Sample Prediction:")
print(f"   True Label: {y_test.iloc[0]}")
print(f"   Predicted Label: {prediction[0]}")
if prediction_proba is not None:
    print(f"   Prediction Probability: Benign={prediction_proba[0][0]:.4f}, Malignant={prediction_proba[0][1]:.4f}")
print(f"   Result: {'✅ Correct' if prediction[0] == y_test.iloc[0] else '❌ Incorrect'}")

## 📝 Step 15: Summary & Conclusions

In [ ]:
print("="*80)
print("🎯 CANCER PREDICTION - OPTIMIZED ML PIPELINE SUMMARY")
print("="*80)
print(f"\n📊 Dataset Information:")
print(f"   Total Samples: {len(cancer)}")
print(f"   Features: {X.shape[1]}")
print(f"   Train/Test Split: {len(X_train)}/{len(X_test)}")
print(f"\n🏆 Best Model: {best_model_name}")
print(f"   Test Accuracy: {best_score:.4f} ({best_score*100:.2f}%)")
print(f"   Precision: {precision:.4f}")
print(f"   Recall: {recall:.4f}")
print(f"   F1-Score: {f1:.4f}")
if y_pred_proba is not None:
    print(f"   AUC-ROC: {auc_score:.4f}")
print(f"\n✅ Key Improvements Implemented:")
print(f"   ✓ Fixed data leakage (scaling after train-test split)")
print(f"   ✓ Cross-validation (5-fold stratified)")
print(f"   ✓ Multiple model comparison ({len(models)} models tested)")
print(f"   ✓ Hyperparameter tuning (GridSearchCV)")
print(f"   ✓ Feature selection & importance analysis")
print(f"   ✓ Comprehensive visualizations (ROC, PR curves, confusion matrices)")
print(f"   ✓ Model persistence (saved for future use)")
print(f"\n💾 Saved Files:")
print(f"   - {model_filename}")
print(f"   - {scaler_filename}")
print("\n" + "="*80)
print("✅ Pipeline completed successfully!")
print("="*80)